In [ ]:
from googletrans import Translator
from transformers import MarianTokenizer, MarianMTModel
import re
from functools import lru_cache

class HinglishTranslator:
    def __init__(self, model_name, word_replacements):
        # Initialize the translator with the transformer model and word replacements
        self.tokenizer = MarianTokenizer.from_pretrained(model_name)
        self.model = MarianMTModel.from_pretrained(model_name)
        self.word_replacements = word_replacements

    @lru_cache(maxsize=None)
    def translate(self, text, use_transformer=True):
        # Used caching to improve translation performance
        if use_transformer:
            return self._translate_with_transformer(text)
        else:
            return self._translate_with_2nd_translator(text)

    def _translate_with_transformer(self, text):
        # Translate text using the transformer model
        input_ids = self.tokenizer.encode(text, return_tensors="pt")
        translated_ids = self.model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
        translated_text = self.tokenizer.decode(translated_ids[0], skip_special_tokens=True)
        return translated_text

    def _translate_with_2nd_translator(self, text):
        # Translate text using 2nd translator in case of bad reviews
        translator = Translator()
        translated = translator.translate(text, src='en', dest='hi')
        return translated.text

    def replace_words(self, text):
        # Replace certain words in the translated text
        # list the most used words in social apps, also useful for phrase conversion
        pattern = re.compile(r'\b(' + '|'.join(map(re.escape, self.word_replacements.keys())) + r')\b', re.IGNORECASE)
        translated_text = pattern.sub(lambda match: self.word_replacements.get(match.group().lower(), match.group()), text)

        for source, target in self.word_replacements.items():
            translated_text = translated_text.replace(source.lower(), target)

        return translated_text

    def translate_and_review(self, text, use_transformer=True):
        # Translate the text and allow for user/employee review
        translated_text = self.translate(text, use_transformer)
        return translated_text

def main():
    # Set up the translator and word replacements
    model_name = "Helsinki-NLP/opus-mt-en-hi"
    word_replacements = {
         "निश्चित रूप": "definitely",
        "टिप्पणी": "comment",
        "कमेंट": "comment",
        "प्रतिक्रिया": "feedback",
        "फीडबैक": "feedback",
        "खण्ड": "section",
        "वीडियो": "video",
        "उत्पादों": "products",
        "बैग": "bag",
        "सिर": "headset",
        "मिनट": "minute",
        "उपयोग": "use",
        "इंतजार": "wait",
        "स्पष्ट रूप से": "clearly",
        "उल्लेख": "mention",
        "इंतज़ार": "wait",
        "सेक्शन": "section",
        "साझा": "share"
    }
    translator = HinglishTranslator(model_name, word_replacements)

    english_texts = [
        "Definitely share your feedback in the comment section.",
        "So even if it's a big video, I will clearly mention all the products.",
        "I was waiting for my bag."
    ]

    max_translation_attempts = 3

    for text in english_texts:
        successful_translation = False
        translation_attempts = 0
        use_transformer = True
        translated_text = text

        # Attempt translation and review based on user input
        while not successful_translation and translation_attempts < max_translation_attempts:
            translated_text = translator.translate_and_review(text, use_transformer)

            print("English:", text)
            print("Hinglish (Translated):", translated_text)

            rating = input("Rate the translation (good/bad/neutral): ")

            if rating == "good":
                successful_translation = True
            else:
                translation_attempts += 1

                if translation_attempts < max_translation_attempts:
                    decision = input("Translation not rated good enough. Switch to 2nd Translator? (yes/no): ")
                    if decision.lower() == "yes":
                        use_transformer = False
        # can be sent again with high size to get the context if needed
        if not successful_translation:
            print("Translation not rated good enough after multiple attempts. Using 2nd Translator.")
            translated_text = translator.translate_and_review(text, use_transformer=False)

        # Apply word replacement after review
        replaced_text = translator.replace_words(translated_text)
        print("Hinglish (After Replacement and Review):", replaced_text)
        print()

if __name__ == "__main__":
    main()
